# Using the Neo4j Graph Data Science Library from Python

In [1]:
from neo4j import GraphDatabase

In [2]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "<YOUR_PASSWORD>"))

## Creating the named projected graph

In [3]:
graphName = "my_complex_projected_graph"

nodeProj = {
    "User": {
        "label": "User",
    }
}

relProj = {
    "FOLLOWS": {
        "type": "FOLLOWS",
        "orientation": "UNDIRECTED",
        "aggregation": "SINGLE"
    }
}

In [4]:
cypher = "CALL gds.graph.create( $graphName, $nodeProj, $relProj)"

In [5]:
with driver.session() as session:
    result = session.run(
        cypher,
        graphName=graphName,
        nodeProj=nodeProj,
        relProj=relProj
    )

In [6]:
result.data()

[{'graphName': 'my_complex_projected_graph',
  'nodeProjection': {'User': {'properties': {}, 'label': 'User'}},
  'relationshipProjection': {'FOLLOWS': {'orientation': 'UNDIRECTED',
    'aggregation': 'SINGLE',
    'type': 'FOLLOWS',
    'properties': {}}},
  'nodeCount': 596,
  'relationshipCount': 1192,
  'createMillis': 8}]

## Running an algorithm: example with PageRank

### Algorithm configuration dict

In [7]:
algoConfig = {
    "dampingFactor": 0.85,
    # "relationshipWeightProperty": "weight"
}

In [8]:
with driver.session() as session:
    result = session.run(
        "CALL gds.pageRank.stream($graphName, $algoConfig)",
        graphName=graphName,
        algoConfig=algoConfig,
    )

In [9]:
for record in result:
    print(record.data())

{'nodeId': 0, 'score': 0.15000000000000002}
{'nodeId': 1, 'score': 1.2104374413844197}
{'nodeId': 2, 'score': 0.9438581566326321}
{'nodeId': 3, 'score': 0.4038160189986229}
{'nodeId': 4, 'score': 0.6520907054888087}
{'nodeId': 5, 'score': 1.733900258783251}
{'nodeId': 6, 'score': 1.970460276398808}
{'nodeId': 7, 'score': 0.4038160189986229}
{'nodeId': 8, 'score': 2.8126600407063966}
{'nodeId': 9, 'score': 0.9132519175764172}
{'nodeId': 10, 'score': 9.010058941878375}
{'nodeId': 11, 'score': 11.484363865479828}
{'nodeId': 12, 'score': 10.334434717893597}
{'nodeId': 13, 'score': 3.51371067268774}
{'nodeId': 14, 'score': 4.429150780197232}
{'nodeId': 15, 'score': 12.88761965315789}
{'nodeId': 16, 'score': 10.417098037712279}
{'nodeId': 17, 'score': 1.8435446158517153}
{'nodeId': 18, 'score': 14.709171251580118}
{'nodeId': 19, 'score': 2.159772182954475}
{'nodeId': 20, 'score': 9.270769257098436}
{'nodeId': 21, 'score': 5.6819858958479035}
{'nodeId': 22, 'score': 10.651387471333145}
{'node

### Using `gds.util.asNode` to get more meaningful results

In [13]:
with driver.session() as session:
    result = session.run(
        """CALL gds.pageRank.stream($graphName, $algoConfig)
            YIELD nodeId, score
            RETURN gds.util.asNode(nodeId) as node, score
        """,
        graphName=graphName,
        algoConfig=algoConfig,
    )

In [14]:
for record in result:
    print(record.get("node").get("user_id"), record.get("score"))

1 0.15000000000000002
31 1.2104374413844197
32 0.9438581566326321
33 0.4038160189986229
34 0.6520907054888087
35 1.733900258783251
36 1.970460276398808
37 0.4038160189986229
38 2.8126600407063966
39 0.9132519175764172
40 9.010058941878375
41 11.484363865479828
42 10.334434717893597
43 3.51371067268774
44 4.429150780197232
45 12.88761965315789
46 10.417098037712279
47 1.8435446158517153
48 14.709171251580118
49 2.159772182954475
50 9.270769257098436
51 5.6819858958479035
52 10.651387471333145
53 4.655745753366499
54 3.9165744459256526
55 10.488682551681997
56 7.839183766581118
57 7.951555732078848
58 1.4411550694378095
59 9.722989419475192
60 13.774939332786015
61 10.313357470557094
62 2.1540201521478597
63 10.885534165613354
64 1.0494503078283743
65 5.135402509756387
66 1.3217056492343546
67 11.624030757509173
68 11.090736489370467
69 1.0808968721888963
80 0.4038160189986229
81 0.4038160189986229
82 0.4038160189986229
83 0.4038160189986229
84 0.664977219211869
85 0.4038160189986229
86 

### Writing results back to the graph

In [15]:
algoConfig["writeProperty"] = "pr"

with driver.session() as session:
    result = session.run(
        "CALL gds.pageRank.write($graphName, $algoConfig)",
        graphName=graphName,
        algoConfig=algoConfig,
    )

In [16]:
result.single().data()

{'nodePropertiesWritten': 596,
 'createMillis': 0,
 'computeMillis': 132,
 'writeMillis': 6,
 'ranIterations': 20,
 'didConverge': False,
 'configuration': {'maxIterations': 20,
  'writeConcurrency': 4,
  'sourceNodes': [],
  'writeProperty': 'pr',
  'relationshipWeightProperty': None,
  'dampingFactor': 0.85,
  'relationshipTypes': ['*'],
  'cacheWeights': False,
  'tolerance': 1e-07,
  'concurrency': 4}}

## Dropping a named projected graph

In [17]:
with driver.session() as session:
    result = session.run(
        "CALL gds.graph.drop($graphName)",
        graphName=graphName,
    )
result.data()

[{'graphName': 'my_complex_projected_graph',
  'nodeProjection': {'User': {'properties': {}, 'label': 'User'}},
  'relationshipProjection': {'FOLLOWS': {'orientation': 'UNDIRECTED',
    'aggregation': 'SINGLE',
    'type': 'FOLLOWS',
    'properties': {}}},
  'nodeQuery': None,
  'relationshipQuery': None,
  'nodeCount': 596,
  'relationshipCount': 1192,
  'degreeDistribution': {'p99': 30,
   'min': 0,
   'max': 48,
   'mean': 2.0,
   'p90': 2,
   'p50': 1,
   'p999': 39,
   'p95': 6,
   'p75': 1}}]